In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
from torch import nn
from torchsummary import summary
import torchvision.models as models

In [3]:
#loading original modelsa
resnet34_original=models.resnet34(pretrained=False)
print(resnet34_original)
summary(resnet34_original, (3,64,64))

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
class BasicBlock(nn.Module):
    expansion=1
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1=nn.Conv2d(in_channels, out_channels, stride=stride, kernel_size=3, padding=1, bias=False)
        self.bn1=nn.BatchNorm2d(out_channels)
        self.conv2=nn.Conv2d(out_channels, out_channels*self.expansion, stride=1, kernel_size=3, padding=1, bias=False)
        self.bn2=nn.BatchNorm2d(out_channels*self.expansion)

        self.relu=nn.ReLU()

        self.downsample=downsample

    def forward(self, x:torch.Tensor) ->torch.Tensor:
        identity=x
        out=self.conv1(x)
        out=self.bn1(out)
        out=self.relu(out)
        out=self.conv2(out)
        out=self.bn2(out)
        if self.downsample is not None:
            identity=self.downsample(x)

        out+=identity
        out=self.relu(out)
        return out

In [5]:
class ResNet34custom(nn.Module):
    def __init__(self):
        super(ResNet34custom, self).__init__()
        self.conv1= nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1=nn.BatchNorm2d(64)
        self.relu=nn.ReLU()
        self.maxpool=nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1=nn.Sequential(BasicBlock(in_channels=64, out_channels=64), 
                                  BasicBlock(in_channels=64, out_channels=64), 
                                  BasicBlock(in_channels=64, out_channels=64))

        dwns1=nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1, stride=2, bias=False), 
                            nn.BatchNorm2d(128))
        
        self.layer2=nn.Sequential(BasicBlock(in_channels=64, out_channels=128, stride=2, downsample=dwns1),
                                  BasicBlock(in_channels=128, out_channels=128),
                                  BasicBlock(in_channels=128, out_channels=128),
                                  BasicBlock(in_channels=128, out_channels=128))
        
        dwns2=nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256, kernel_size=1, stride=2, bias=False),
                          nn.BatchNorm2d(256))
        self.layer3=nn.Sequential(BasicBlock(in_channels=128, out_channels=256, stride=2, downsample=dwns2),
                                  BasicBlock(in_channels=256, out_channels=256),
                                  BasicBlock(in_channels=256, out_channels=256),
                                  BasicBlock(in_channels=256, out_channels=256),
                                  BasicBlock(in_channels=256, out_channels=256),
                                  BasicBlock(in_channels=256, out_channels=256))
        
        dwns3=nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=1, stride=2, bias=False),
                          nn.BatchNorm2d(512))
        self.layer4=nn.Sequential(BasicBlock(in_channels=256, out_channels=512, stride=2, downsample=dwns3),
                                  BasicBlock(in_channels=512, out_channels=512),
                                  BasicBlock(in_channels=512, out_channels=512))
        
        self.avgpool=nn.AdaptiveAvgPool2d((1,1))
        self.fc=nn.Linear(in_features=512, out_features=1000)
        
        

    def forward(self, x:torch.Tensor) ->torch.Tensor:
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu(x)
        x=self.maxpool(x)
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        x=self.avgpool(x)
        x=torch.flatten(x,1)
        x=self.fc(x)
        return x

In [6]:
resnet34_custom=ResNet34custom()
summary(resnet34_custom, (3,64,64))
print(resnet34_custom)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           9,408
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          36,864
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,864
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
       BasicBlock-11           [-1, 64, 16, 16]               0
           Conv2d-12           [-1, 64, 16, 16]          36,864
      BatchNorm2d-13           [-1, 64, 16, 16]             128
             ReLU-14           [-1, 64,